In [1]:

from analysis.probability import cumulative_average
from evaluate import evaluate_single

probability_generator = cumulative_average
tau = 0.001

def eval_simple(note, tau=tau):
    return evaluate_single(probability_generator(note), root_note=note, tau=tau)

In [2]:
import numpy as np
from analysis.musical import consonance_ordered_notes, empirical_probabilities
from analysis.spike_tensor import generate_spike_tensor
from cache import get_spikes
from evaluate import predicted_consonance_scores, predicted_probabilities

root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes_spikes = [get_spikes(note) for note in notes]
consonance_ordered_tensors = [generate_spike_tensor(spikes, tau=tau) for spikes in notes_spikes]

root_tensor = generate_spike_tensor(get_spikes(root_note), tau=tau)
probability_tensor = probability_generator(root_note)

def try_eval(tensors, metric, debug=False):
    scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)

    P_predicted = predicted_probabilities(scores)

    if debug:
        def print_arr(arr):
            print(np.round(np.array(arr), 3))
        print("Scores")
        print_arr(scores)
        print("P_predicted")
        print_arr(P_predicted)
        print("P_empirical")
        print_arr(empirical_probabilities())

    print(metric(P_predicted))

In [3]:
from evaluate import js_divergence

try_eval(consonance_ordered_tensors, js_divergence, debug=True)

Scores
[-0.339 -0.389 -0.392 -0.396 -0.395 -0.374 -0.386 -0.386 -0.383 -0.359
 -0.385 -0.382]
P_predicted
[0.309 0.038 0.022 0.    0.003 0.117 0.051 0.056 0.071 0.202 0.057 0.074]
P_empirical
[0.147 0.133 0.12  0.107 0.107 0.093 0.08  0.067 0.053 0.053 0.027 0.013]
0.3948532616795133


In [4]:
import numpy as np

def shuffle_attempts(metric, N=10):
    data = []
    for i in range(N):
        A = consonance_ordered_tensors[0][0][0]
        np.random.shuffle(consonance_ordered_tensors)
        B = consonance_ordered_tensors[0][0][0]
        if not np.array_equal(A, B):
            value = try_eval(consonance_ordered_tensors, metric)
            data.append(value)

In [5]:
from evaluate import js_divergence


shuffle_attempts(js_divergence)

0.31400770000425265
0.3504222029179894
0.42177580960791405
0.3734869980285632
0.42052838972775375
0.4516911211357512
0.4161585553811662
0.44809622950686584
0.3809541723386174


In [6]:
import os

from cache import SPIKES_DIR


root_note = "C4"
notes = consonance_ordered_notes(root_note)
notes = [notes[0], notes[-1]]
notes_spikes = [get_spikes(note) for note in notes]
tensors = [generate_spike_tensor(spikes, tau=tau) for spikes in notes_spikes]

white_noise_spikes = np.load(os.path.join(SPIKES_DIR, "white_noise.npy"))
tensors.append(generate_spike_tensor(white_noise_spikes, tau=tau))

root_tensor = generate_spike_tensor(get_spikes(root_note), tau=tau)

scores = predicted_consonance_scores(probability_tensor, tensors, root_tensor)
print("Scores")
print(np.round(np.array(scores), 2))

P_predicted = predicted_probabilities(scores)
print("P_predicted")
print(np.round(np.array(P_predicted), 2))

print("Most Cons, Least Cons, White Noise")


Scores
[-0.34 -0.38 -0.61]
P_predicted
[0.54 0.46 0.  ]
Most Cons, Least Cons, White Noise


In [7]:
keys = ["C4", "C#4", "D4", "D#4", "E4", "F4", "F#4", "G4", "G#4", "A4", "A#4", "B4"]

for key in keys:
    print(f"{key}: {eval_simple(key)}")

C4: 61
C#4: 59
D4: 62
D#4: 65
E4: 68
F4: 61
F#4: 63
G4: 69
G#4: 68
A4: 71
A#4: 62
B4: 75


In [8]:
consonance_ordered_notes(root_note)

['C5', 'G4', 'F4', 'E4', 'D#4', 'A4', 'G#4', 'D4', 'C#4', 'B4', 'A#4', 'F#4']

In [9]:
# shuffle_attempts(kl_divergence_reverse)
# shuffle_attempts(kl_divergence)